#  Seasonal auto regressive integrated moving average with exogenous regressors model

In [19]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
import math
from common.utils import mape, load_data
import os
import pandas as pd
import numpy as np

In [4]:
# os.chdir("../../")
data_dir = "data"
# Load the data from csv into a pandas dataframe
ts_data_load = load_data(data_dir)[['load']]
ts_data_load.head()

,load
2012-01-01 00:00:00,2698.0
2012-01-01 01:00:00,2558.0
2012-01-01 02:00:00,2444.0
2012-01-01 03:00:00,2402.0
2012-01-01 04:00:00,2403.0


In [6]:
train_start_dt = '2014-11-01 00:00:00'
test_start_dt = '2014-12-30 00:00:00'
train = ts_data_load.copy() [(ts_data_load.index >= train_start_dt) & (ts_data_load.index < test_start_dt)][['load']]
test = ts_data_load.copy()[ts_data_load.index >= test_start_dt][['load']]

In [7]:
print('Train data shape: ', train.shape)
print('Test data shape: ', test.shape)

Train data shape:  (1416, 1)
Test data shape:  (48, 1)


___

# Sanitization

In [8]:
# Scale train data to be in range (0, 1)
scaler = MinMaxScaler()
train['load'] = scaler.fit_transform(train)
train.head()
# Scale test data to be in range (0, 1)
test['load'] = scaler.transform(test)
test.head()

,load
2014-12-30 00:00:00,0.329454
2014-12-30 01:00:00,0.290063
2014-12-30 02:00:00,0.273948
2014-12-30 03:00:00,0.268129
2014-12-30 04:00:00,0.302596


--- 
# Specify the number of steps to forecast ahead

In [10]:
HORIZON = 3
print('Forecasting horizon:', HORIZON, 'hours')

Forecasting horizon: 3 hours


In [11]:
# Define the order and seasonal order for the SARIMAX model

In [12]:
order = (4, 1, 0)
seasonal_order = (1, 1, 0, 24)

In [13]:
# Build and fit the SARIMAX model
model = SARIMAX(endog=train, order=order, seasonal_order=seasonal_order)
results = model.fit()
print(results.summary())

                                     SARIMAX Results                                      
Dep. Variable:                               load   No. Observations:                 1416
Model:             SARIMAX(4, 1, 0)x(1, 1, 0, 24)   Log Likelihood                3477.226
Date:                            Sun, 07 Jan 2024   AIC                          -6942.451
Time:                                    03:12:11   BIC                          -6911.025
Sample:                                11-01-2014   HQIC                         -6930.700
                                     - 12-29-2014                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.8392      0.016     52.417      0.000       0.808       0.871
ar.L2         -0.5184      0.034   

In [14]:
# Create a test data point for each HORIZON step
test_shifted = test.copy()
for t in range(1, HORIZON):
    test_shifted['load+'+str(t)] = test_shifted['load'].shift(-t, freq='H')

test_shifted = test_shifted.dropna(how='any')

In [15]:
test_shifted

,load,load+1,load+2
2014-12-30 00:00:00,0.329454,0.290063,0.273948
2014-12-30 01:00:00,0.290063,0.273948,0.268129
2014-12-30 02:00:00,0.273948,0.268129,0.302596
2014-12-30 03:00:00,0.268129,0.302596,0.408236
2014-12-30 04:00:00,0.302596,0.408236,0.568935
2014-12-30 05:00:00,0.408236,0.568935,0.679946
2014-12-30 06:00:00,0.568935,0.679946,0.730976
2014-12-30 07:00:00,0.679946,0.730976,0.751119
2014-12-30 08:00:00,0.730976,0.751119,0.763653
2014-12-30 09:00:00,0.751119,0.763653,0.738138


In [17]:
%%time
# Make predictions on the test data
training_window = 720
train_ts = train['load']
test_ts = test_shifted
history = [x for x in train_ts]
history = history[(-training_window):]
predictions = list()
# Let's user simpler model
order = (2, 1, 0)
seasonal_order = (1, 1, 0, 24)
for t in range(test_ts.shape[0]):
    model = SARIMAX(endog=history, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit()
    yhat = model_fit.forecast(steps = HORIZON)
    predictions.append(yhat)
    obs = list(test_ts.iloc[t])
    # move the training window
    history.append(obs[0])
    history.pop(0)
    print(test_ts.index[t])
    print(t+1, ': predicted =', yhat, 'expected =', obs)

2014-12-30 00:00:00
1 : predicted = [0.32323448 0.28797267 0.27872803] expected = [0.32945389435989236, 0.2900626678603402, 0.2739480752014323]
2014-12-30 01:00:00
2 : predicted = [0.29926113 0.29130843 0.30229186] expected = [0.2900626678603402, 0.2739480752014323, 0.26812891674127126]
2014-12-30 02:00:00
3 : predicted = [0.27449932 0.2834967  0.32317733] expected = [0.2739480752014323, 0.26812891674127126, 0.3025962399283795]
2014-12-30 03:00:00
4 : predicted = [0.28240227 0.3217418  0.41662755] expected = [0.26812891674127126, 0.3025962399283795, 0.40823634735899716]


C:\Users\nishaalnaseer\Documents\pythonWork\time-series\analysis\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


2014-12-30 04:00:00
5 : predicted = [0.29544108 0.38698565 0.54017801] expected = [0.3025962399283795, 0.40823634735899716, 0.5689346463742166]
2014-12-30 05:00:00
6 : predicted = [0.40003586 0.55452886 0.66473958] expected = [0.40823634735899716, 0.5689346463742166, 0.6799462846911368]
2014-12-30 06:00:00
7 : predicted = [0.56931517 0.68132591 0.75044255] expected = [0.5689346463742166, 0.6799462846911368, 0.7309758281110115]
2014-12-30 07:00:00
8 : predicted = [0.68084917 0.75026719 0.79931067] expected = [0.6799462846911368, 0.7309758281110115, 0.7511190689346463]
2014-12-30 08:00:00
9 : predicted = [0.74849754 0.79720209 0.81744496] expected = [0.7309758281110115, 0.7511190689346463, 0.7636526410026856]


C:\Users\nishaalnaseer\Documents\pythonWork\time-series\analysis\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


2014-12-30 09:00:00
10 : predicted = [0.76510008 0.78103988 0.78317492] expected = [0.7511190689346463, 0.7636526410026856, 0.7381378692927483]


...

2014-12-31 06:00:00
31 : predicted = [0.6258175  0.73358771 0.7869518 ] expected = [0.6145926589077886, 0.7247090420769919, 0.786034019695613]
2014-12-31 07:00:00
32 : predicted = [0.71296407 0.76358696 0.7916839 ] expected = [0.7247090420769919, 0.786034019695613, 0.8012533572068039]


C:\Users\nishaalnaseer\Documents\pythonWork\time-series\analysis\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


2014-12-31 08:00:00
33 : predicted = [0.78510517 0.81597403 0.83111054] expected = [0.786034019695613, 0.8012533572068039, 0.7994628469113696]
2014-12-31 09:00:00
34 : predicted = [0.81761421 0.83299109 0.81058757] expected = [0.8012533572068039, 0.7994628469113696, 0.780214861235452]
2014-12-31 10:00:00
35 : predicted = [0.80285383 0.77645504 0.75628776] expected = [0.7994628469113696, 0.780214861235452, 0.7587287376902416]
2014-12-31 11:00:00
36 : predicted = [0.77019234 0.74919319 0.74258019] expected = [0.780214861235452, 0.7587287376902416, 0.7367949865711727]
2014-12-31 12:00:00
37 : predicted = [0.76754932 0.76339668 0.76457003] expected = [0.7587287376902416, 0.7367949865711727, 0.7188898836168307]
2014-12-31 13:00:00
38 : predicted = [0.74719006 0.74621783 0.77528239] expected = [0.7367949865711727, 0.7188898836168307, 0.7273948075201431]
2014-12-31 14:00:00
39 : predicted = [0.72707958 0.75359712 0.86811124] expected = [0.7188898836168307, 0.7273948075201431, 0.82990152193375

C:\Users\nishaalnaseer\Documents\pythonWork\time-series\analysis\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


2014-12-31 19:00:00
44 : predicted = [0.78909233 0.73038993 0.62943926] expected = [0.7721575649059982, 0.7023276633840643, 0.6195165622202325]
2014-12-31 20:00:00
45 : predicted = [0.69911683 0.5938633  0.4597674 ] expected = [0.7023276633840643, 0.6195165622202325, 0.5425246195165621]
2014-12-31 21:00:00
46 : predicted = [0.59942945 0.46565987 0.35597868] expected = [0.6195165622202325, 0.5425246195165621, 0.4735899731423454]
CPU times: total: 25.7 s
Wall time: 2min 29s


In [20]:
# Compare predictions to actual load
eval_df = pd.DataFrame(predictions,
                       columns=['t+'+str(t) for t in range(1, HORIZON+1)])
eval_df['timestamp'] = test.index[0:len(test.index)-HORIZON+1]
eval_df = pd.melt(
    eval_df, id_vars='timestamp',
    value_name='prediction', var_name='h'
)
eval_df['actual'] = np.array(np.transpose(test_ts)).ravel()
eval_df[['prediction', 'actual']] = scaler.inverse_transform(eval_df[['prediction', 'actual']])

In [21]:
# Compute the mean absolute percentage error (MAPE)
if(HORIZON > 1):
    eval_df['APE'] = (eval_df['prediction'] -
                      eval_df['actual']).abs() / eval_df['actual']
    print(eval_df.groupby('h')['APE'].mean())

h
t+1    0.005570
t+2    0.012086
t+3    0.016755
Name: APE, dtype: float64


In [23]:
# Print one-step forecast MAPE
print(
    "One-step forecast MAPE: ",
    (
        mape(
            eval_df[eval_df["h"] == "t+1"]["prediction"],
            eval_df[eval_df["h"] == "t+1"]["actual"],
        )
    )
    * 100,
    "%",
)
print(
    "Multi-step forecast MAPE: ",
    mape(eval_df["prediction"], eval_df["actual"]) * 100,
    "%",
)

One-step forecast MAPE:  0.5570465121546399 %
Multi-step forecast MAPE:  1.1470670530378426 %
